# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [ ]:
# Load the dataset
file_path = '/mnt/data/marketing.csv'
df = pd.read_csv(file_path)

# Display the first 5 rows
print(df.head())

# Check the shape of the dataset
print(df.shape)

# Check the general information of the dataset
print(df.info())

### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [ ]:
# Check the percentage of missing values
missing_values = df.isnull().mean() * 100
print(missing_values)

# If there are missing values, drop or fill them
df = df.dropna()  # This line drops any rows with missing values

### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [ ]:
# Check for duplicate records
duplicates = df.duplicated().sum()
print(f"Number of duplicate records: {duplicates}")

# Drop duplicate records
df = df.drop_duplicates()

# Check the shape of the dataframe after dropping duplicates
print(df.shape)

### 5. Drop the columns which you think redundant for the analysis 

In [ ]:

# Drop columns that are redundant (you can adjust this based on your knowledge of the dataset)
df = df.drop(columns=['ID'])  # Example of dropping a column

### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [ ]:
# Check unique categories in 'Marital_Status'
unique_categories = df['Marital_Status'].unique()
print(unique_categories)

# Group categories
df['Marital_Status'] = df['Marital_Status'].replace(
    {'Married': 'relationship', 'Together': 'relationship',
     'Divorced': 'Single', 'Widow': 'Single', 'Alone': 'Single',
     'YOLO': 'Single', 'Absurd': 'Single'}
)

### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [ ]:
# Group columns into 'Total_Expenses'
df['Total_Expenses'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'

In [ ]:
# Group columns into 'Num_Total_Purchases'
df['Num_Total_Purchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases']].sum(axis=1)

### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [ ]:
# Group columns into 'Kids'
df['Kids'] = df[['Kidhome', 'Teenhome']].sum(axis=1)

### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [ ]:
# Group columns into 'TotalAcceptedCmp'
df['TotalAcceptedCmp'] = df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']].sum(axis=1)

### 11. Drop those columns which we have used above for obtaining new features

In [ ]:
# Drop the original columns used for grouping
df = df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
                      'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases',
                      'Kidhome', 'Teenhome', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4',
                      'AcceptedCmp5', 'Response'])

### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [ ]:
# Extract 'age' and drop 'Year_Birth'
df['Age'] = 2024 - df['Year_Birth']
df = df.drop(columns=['Year_Birth'])

### 13. Encode the categorical variables in the dataset

In [ ]:
# Encode categorical variables
df = pd.get_dummies(df, drop_first=True)

### 14. Standardize the columns, so that values are in a particular range

In [ ]:
# Standardize numerical columns
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
# Apply PCA
pca = PCA()
pca.fit(df_scaled)
explained_variance = np.cumsum(pca.explained_variance_ratio_) * 100

# Determine number of components for 90-95% variance
n_components = np.argmax(explained_variance >= 90) + 1
print(f"Number of components explaining 90% variance: {n_components}")

# Apply PCA with determined number of components
pca = PCA(n_components=n_components)
df_pca = pca.fit_transform(df_scaled)

### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [ ]:
# Apply K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(df_pca)

# Add cluster labels to the dataframe
df['Cluster_KMeans'] = kmeans_labels

### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [ ]:
# Apply Agglomerative clustering
agg_clustering = AgglomerativeClustering(n_clusters=5)
df['Agg_Cluster'] = agg_clustering.fit_predict(df_scaled)

# Perform cluster analysis
for col in df.columns:
    if col not in ['PCA_Cluster', 'Agg_Cluster']:
        plt.figure(figsize=(10, 5))
        sns.boxplot(x='Agg_Cluster', y=col, data=df)
        plt.title(f'Cluster analysis for {col}')
        plt.show()

### Visualization and Interpretation of results

-----
## Happy Learning
-----